In [68]:
import pandas as pd
import numpy as np

In [69]:
YEAR = 2023
TARGET = 'customer'
PATH = f'../dataset/imbank_{TARGET.lower()}_data/iMBANK_{TARGET.upper()}_DATA_{YEAR}(K-DigitalTraining).csv'
df = pd.read_csv(PATH, encoding='euc-kr')

In [70]:
customer = df[['기준년월', '고객ID']]
customer=customer.drop_duplicates()
a1 = customer.groupby('고객ID')['기준년월'].min()
a2 = customer.groupby('고객ID')['기준년월'].max()
customer=pd.merge(a1, a2, how='left', on='고객ID')
customer.columns = ['최초', '최후']
del a1, a2
customer = customer.reset_index()
case1 = customer[(customer['최초'] == int(f'{YEAR}01')) & (customer['최후'] == int(f'{YEAR}12'))] # 관찰대상 12개월 다 존재
# 이탈
case2 = customer[(customer['최초'] == int(f'{YEAR}01')) & (customer['최후'] != int(f'{YEAR}12'))]
# 신규가입
case3 = customer[(customer['최초'] != int(f'{YEAR}01')) & (customer['최후'] == int(f'{YEAR}12'))]

ids = np.concatenate((case1['고객ID'],case2['고객ID'],case3['고객ID']))
# 가입 및 이탈 (이탈로로)
case4=customer[~customer['고객ID'].isin(ids)]

In [71]:
# 신규ID, 이탈ID
ids_new=case3['고객ID']

In [72]:
ids_leave =  np.concatenate((case2['고객ID'],case4['고객ID']))

In [73]:
# 1. '지역구분' 컬럼을 0으로 초기화
df['지역구분'] = 0

# 2. '자택_시도'가 '대구광역시' 또는 '경상북도'인 경우 '지역구분'을 1로 설정
df.loc[df['자택_시도'].isin(['대구광역시', '경상북도']), '지역구분'] = 1

In [74]:
df[df['고객ID'].isin(ids_new)].drop_duplicates('고객ID')['지역구분'].value_counts()

지역구분
0    98737
1    29375
Name: count, dtype: int64

In [75]:
df[df['고객ID'].isin(ids_leave)].drop_duplicates('고객ID')['지역구분'].value_counts().sort_index()

지역구분
0     6180
1    16256
Name: count, dtype: int64

In [76]:
df[df['고객ID'].isin(ids_new)].drop_duplicates('고객ID')['연령대'].value_counts().sort_index()

연령대
10대미만     6100
20대      36123
30대      30336
40대      26237
50대      18491
60대이상    10825
Name: count, dtype: int64

In [77]:
df[df['고객ID'].isin(ids_leave)].drop_duplicates('고객ID')['연령대'].value_counts().sort_index()

연령대
10대미만      15
20대       952
30대      4422
40대      5641
50대      5790
60대이상    5616
Name: count, dtype: int64

In [78]:
df[df['고객ID'].isin(ids_leave)].drop_duplicates('고객ID')['자택_시도'].value_counts().sort_index()

자택_시도
강원특별자치도       30
경기도         2628
경상남도         292
경상북도        3695
광주광역시         23
대구광역시      12561
대전광역시        472
부산광역시        446
서울특별시       1033
세종특별자치시       93
울산광역시        153
인천광역시        372
전라남도          16
전북특별자치도       13
제주특별자치도       18
충청남도          63
충청북도         528
Name: count, dtype: int64

In [79]:
df[df['고객ID'].isin(ids_new)].drop_duplicates('고객ID')['자택_시도'].value_counts().sort_index()

자택_시도
강원특별자치도     2595
경기도        30389
경상남도        6433
경상북도       11193
광주광역시       3081
대구광역시      18182
대전광역시       3204
부산광역시       7570
서울특별시      21450
세종특별자치시      574
울산광역시       2159
인천광역시       7932
전라남도        2267
전북특별자치도     2427
제주특별자치도     1489
충청남도        3997
충청북도        3170
Name: count, dtype: int64